# **Predictive Modeling for Rocket Landing Success**  
### *A Machine Learning Approach Using SpaceX Falcon 9 Data* 

## **Data Collection API**

### Installing Essential Python Libraries

**`requests`**: Simplifies sending HTTP requests.

**`pandas`**: Tools for data analysis and manipulation.

**`numpy`**: Library for working with arrays and performing mathematical operations.

In [ ]:
!pip install requests 
!pip install pandas
!pip install numpy

### Import Libraries

In [2]:
import requests
import pandas as pd
import numpy as np

import datetime
# Why No `pip install` for `datetime`?
# It's a standard Python library, which means it is included with Python by default. 
# There's no need to install it using `pip`, just import it directly and use it.


### Setting to Display All Columns And Data in a Dataframe

In [5]:
# Setting this option will print all columns of a dataframe
pd.set_option('display.max_columns', None)

# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)